In [19]:
%pip install numpy
%pip install pandas
%pip install matplotlib
%pip install seaborn

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

In [21]:
results_dataset = "./results/2025AC-DL.csv"

df = pd.read_csv(results_dataset)
df['total_votes'] = df['evm_votes'] + df['postal_votes']
display(df)

,state,constituency,serial_no,candidate,party,evm_votes,postal_votes,total_votes
0,NCT of Delhi,NERELA,1,ARUNA,Indian National Congress,6724,58,6782
1,NCT of Delhi,NERELA,2,RAJ KARAN KHATRI,Bharatiya Janata Party,86463,752,87215
2,NCT of Delhi,NERELA,3,SHARAD KUMAR,Aam Aadmi Party,78395,224,78619
3,NCT of Delhi,NERELA,4,ANIL KUMAR SINGH,Communist Party of India (Marxist-Leninist) (L...,328,0,328
4,NCT of Delhi,NERELA,5,MD KHALID,Aazad Samaj Party (Kanshi Ram),278,0,278
...,...,...,...,...,...,...,...,...
764,NCT of Delhi,KARAWAL NAGAR,12,H.M. HASHIM MALIK,Social Democratic Party Of India,491,0,491
765,NCT of Delhi,KARAWAL NAGAR,13,AJAY SINGH NEGI,Independent,1002,4,1006
766,NCT of Delhi,KARAWAL NAGAR,14,YOGESH SWAMY,Independent,228,1,229
767,NCT of Delhi,KARAWAL NAGAR,15,VISHAL SHARMA,Independent,173,0,173


In [22]:
df['party'].value_counts()

party
Independent                             138
Indian National Congress                 70
Aam Aadmi Party                          70
None of the Above                        70
Bharatiya Janata Party                   68
                                       ... 
One Bharat Citizen Party                  1
Proutist Bloc, India                      1
Rashtravadi Loktantrik Party (India)      1
Bhartiya Krishak Dal                      1
Social Democratic Party Of India          1
Name: count, Length: 107, dtype: int64

In [23]:
# Get the winning candidate from each constituency
winning_candidates = df.loc[df.groupby('constituency')['total_votes'].idxmax()]
winning_candidates = winning_candidates[['constituency', 'party', 'candidate', 'total_votes']]
winning_candidates = winning_candidates.sort_values('total_votes', ascending=False).reset_index(drop=True)
display(winning_candidates.head(25))

,constituency,party,candidate,total_votes
0,MATIALA,Bharatiya Janata Party,SANDEEP SEHRAWAT,146295
1,VIKASPURI,Bharatiya Janata Party,PANKAJ KUMAR SINGH,135564
2,BURARI,Aam Aadmi Party,SANJEEV JHA,121181
3,BAWANA,Bharatiya Janata Party,RAVINDER INDRAJ SINGH,119515
4,BADARPUR,Aam Aadmi Party,RAM SINGH NETAJI,112991
5,KARAWAL NAGAR,Bharatiya Janata Party,KAPIL MISHRA,107367
6,KIRARI,Aam Aadmi Party,ANIL JHA,105780
7,RITHALA,Bharatiya Janata Party,KULWANT RANA,104371
8,UTTAM NAGAR,Bharatiya Janata Party,PAWAN SHARMA,103613
9,NAJAFGARH,Bharatiya Janata Party,NEELAM PAHALWAN,101708


In [31]:
total_votes_cast = df['total_votes'].sum()
parties_by_vote_share = df.groupby('party')['total_votes'].sum().sort_values(ascending=False).reset_index()
parties_by_vote_share['vote_share'] = round(parties_by_vote_share['total_votes'] / total_votes_cast * 100, 2)
nota_index = parties_by_vote_share[parties_by_vote_share['party'] == 'None of the Above'].index[0]
display(parties_by_vote_share.loc[:9 if nota_index < 10 else nota_index])

,party,total_votes,vote_share
0,Bharatiya Janata Party,4323110,45.56
1,Aam Aadmi Party,4133898,43.57
2,Indian National Congress,601922,6.34
3,Janata Dal (United),100580,1.06
4,All India Majlis-E-Ittehadul Muslimeen,73032,0.77
5,Bahujan Samaj Party,55066,0.58
6,None of the Above,53738,0.57
7,Lok Janshakti Party(Ram Vilas),50209,0.53
8,Independent,43833,0.46
9,Aazad Samaj Party (Kanshi Ram),8727,0.09


In [17]:
df[df['party'] == 'None of the Above'].index

Index([  7,  22,  35,  50,  63,  75,  83,  97, 107, 120, 130, 137, 146, 156,
       164, 171, 181, 191, 203, 211, 223, 233, 240, 246, 259, 269, 278, 289,
       296, 313, 327, 337, 351, 365, 375, 388, 396, 404, 412, 436, 446, 452,
       460, 474, 483, 495, 503, 513, 525, 532, 546, 556, 569, 584, 592, 602,
       612, 628, 639, 647, 654, 665, 677, 693, 707, 717, 726, 739, 752, 768],
      dtype='int64')

In [ ]:
df.drop(['election_year', 'election_type'], axis=1, inplace=True)
df

In [ ]:
df[['constituency', 'candidate', 'party', 'total_votes']].sort_values(by='total_votes', ascending=False).groupby('constituency').agg(list)

In [ ]:
sc = pd.read_csv('states.csv')
sc.rename(columns={'code': 'election_state'}, inplace=True)

sc

In [ ]:
dfs = pd.concat([pd.read_csv('2024AC-HR.csv'), 
                 pd.read_csv('2024AC-JH.csv'), 
                 pd.read_csv('2024AC-JK.csv'), 
                 pd.read_csv('2024AC-MH.csv')], ignore_index=True)
dfs = dfs.merge(sc, on='election_state', how='left')
dfs

In [ ]:
dfs['total_votes'] = dfs['evm_votes'] + dfs['postal_votes']
dfs.groupby('party')['total_votes'].sum().sort_values(ascending=False).head(10)